In [151]:
import os
import json

#load data
file_name = "./data/actual.json"

with open(file_name, 'r') as file:
    data = json.load(file) 
data=data[0:1000]

In [152]:
#find vector dimensions for occurrence of city and merchandise in one route so all 
#cities and merchandise characterize a route
vector=set()
for route in data:
    for trip in route["route"]:
        vector.update([trip["from"],trip["to"]])
        for merch in trip["merchandise"]:
            vector.update([merch])
print(list(vector)[:10])
print(len(vector))  

['Bananas', 'Potatoes', 'Pasta', 'Bologna', 'Milano', 'Siena', 'Pens', 'Cheese', 'Beer', 'Chocolate']
30


In [153]:
#find vector dimensions for combination of cities (so a connection between cities)
#and occurrence of a specific merchandise in the entire route
vector=set()
for route in data:
    for trip in route["route"]:
        vector.update([trip["from"]+"-"+trip["to"]])
        for merch in trip["merchandise"]:
            vector.update([merch])
print(list(vector)[:10])
print(len(vector))  

['Verona-Lecce', 'Lecce-Bolzano', 'Bologna-Verona', 'Rome-Bologna', 'Pens', 'Bologna-Rome', 'Padua-Milano', 'Tomatoes', 'Siena-Verona', 'Padua-Bologna']
79


In [154]:
#find vector dimensions for combination of cities (so a connection between cities) in
#combination with the merchandise that is carried on that route
vector=set()
for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            vector.update([trip["from"]+"-"+trip["to"]+"-"+merch])
print(list(vector)[:10])
print(len(vector))  

['Verona-Genova-Pasta', 'Trento-Rome-Bread', 'Lecce-Padua-Butter', 'Siena-Trento-Chocolate', 'Genova-Lecce-Fish', 'Padua-Rome-Water', 'Trento-Padua-Rice', 'Milano-Verona-Rice', 'Milano-Trento-Cheese', 'Trento-Bologna-Cheese']
620


In [155]:
#find vector dimensions for combination of cities (so a connection between cities) in
#combination with the merchandise and the amount that is carried on that route
vector=set()
for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+str(trip["merchandise"][merch])])
print(list(vector)[:10])
print(len(vector))

['Padua-Milano-Apples-13', 'Trento-Padua-Tea-30', 'Verona-Bolzano-Water-12', 'Lecce-Padua-Honey-4', 'Lecce-Padua-Fish-22', 'Bolzano-Rome-Fish-12', 'Lecce-Padua-Butter-22', 'Genova-Rome-Meat-22', 'Trento-Bologna-Beer-28', 'Trento-Rome-Meat-30']
1380


In [161]:
#find vector dimensions for combination of cities (so a connection between cities) in
#combination with the merchandise and the amount that is carried on that route where 
#the amount is specified by a indicator (small, medium, large)
#to do that the max and min is found for every merchandise
mapminmax={}
possiblecomb={}
vector=set()

for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            if trip["from"]+"-"+trip["to"]+"-"+merch+"-min" in mapminmax:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-min"]=min(mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-min"], trip["merchandise"][merch])
            else:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-min"]= trip["merchandise"][merch]
            if trip["from"]+"-"+trip["to"]+"-"+merch+"-max" in mapminmax:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-max"]=max(mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-max"], trip["merchandise"][merch])
            else:
                mapminmax[trip["from"]+"-"+trip["to"]+"-"+merch+"-max"]= trip["merchandise"][merch]
            
            if trip["from"]+"-"+trip["to"]+"-"+merch in possiblecomb:
                possiblecomb[trip["from"]+"-"+trip["to"]+"-"+merch]=possiblecomb[trip["from"]+"-"+trip["to"]+"-"+merch]+1
            else:
                possiblecomb[trip["from"]+"-"+trip["to"]+"-"+merch]=1

mapborders={}
for item in possiblecomb:
    if possiblecomb[item]>1:
        smallmediumborder=mapminmax[item+"-"+"min"]+((1/3)*(mapminmax[item+"-"+"max"]-mapminmax[item+"-"+"min"]))
        mediumlargeborder=mapminmax[item+"-"+"min"]+((2/3)*(mapminmax[item+"-"+"max"]-mapminmax[item+"-"+"min"]))
        mapborders[item]=[smallmediumborder, mediumlargeborder]
    else:
        mapborders[item]=[0]


for route in data:
    for trip in route["route"]:
        for merch in trip["merchandise"]:
            if mapborders[trip["from"]+"-"+trip["to"]+"-"+merch][0]==0:
                vector.update([trip["from"]+"-"+trip["to"]+"-"+merch])
            else:
                if trip["merchandise"][merch]<mapborders[trip["from"]+"-"+trip["to"]+"-"+merch][0]:
                    vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+"small"])
                elif trip["merchandise"][merch]>mapborders[trip["from"]+"-"+trip["to"]+"-"+merch][1]:
                    vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+"large"])
                else:
                    vector.update([trip["from"]+"-"+trip["to"]+"-"+merch+"-"+"medium"])

print(list(vector)[:20])
print(len(vector))
print(possiblecomb)


my_set = vector
prefix_to_search = "Bolzano-Rome-Fish"

# Check if any element in the set starts with the specified prefix
matching_elements = [element for element in my_set if element.startswith(prefix_to_search)]

if matching_elements:
    print(f"Found elements in the set starting with '{prefix_to_search}': {matching_elements}")
else:
    print(f"No elements found in the set starting with '{prefix_to_search}'.")


['Trento-Verona-Tea-large', 'Trento-Rome-Bread', 'Padua-Milano-Milk-large', 'Lecce-Rome-Bread-large', 'Milano-Trento-Cheese', 'Trento-Padua-Rice', 'Milano-Verona-Rice', 'Lecce-Verona-Yogurt-small', 'Trento-Rome-Milk-medium', 'Lecce-Padua-Pens-small', 'Trento-Lecce-Milk-large', 'Rome-Milano-Yogurt', 'Genova-Verona-Rice', 'Lecce-Bologna-Tomatoes-small', 'Padua-Milano-Potatoes-large', 'Milano-Padua-Chocolate', 'Milano-Verona-Meat-large', 'Trento-Lecce-Rice-large', 'Siena-Lecce-Cheese-medium', 'Bologna-Rome-Beer']
1010
{'Milano-Genova-Tea': 849, 'Milano-Genova-Meat': 849, 'Milano-Genova-Pasta': 849, 'Milano-Genova-Water': 798, 'Genova-Siena-Pasta': 865, 'Genova-Siena-Pens': 878, 'Genova-Siena-Meat': 869, 'Genova-Siena-Honey': 876, 'Genova-Siena-Butter': 868, 'Genova-Siena-Bread': 819, 'Siena-Trento-Carrots': 884, 'Siena-Trento-Honey': 885, 'Siena-Trento-Tomatoes': 830, 'Siena-Trento-Rice': 886, 'Siena-Trento-Apples': 890, 'Trento-Lecce-Pasta': 764, 'Trento-Lecce-Fish': 709, 'Trento-Lecce-A